insert data
* insert

* create
* insert_one
* insert_many
* save

get data
* find
  * to_list
  * first_or_none
  
* get / find_one
* find_many
* find_all

sync


when we would like to create application 

we want to create for users

that why our first class will be user

pydantic + beanie = ❤️


In [347]:
from pydantic import BaseModel


class User(BaseModel):
    name: str
    surname: str
    email: str

type(User)

pydantic._internal._model_construction.ModelMetaclass

In [348]:
not_hot_adam = User(name="Adam", surname="Brzyzek", email="nothotadam@gmail.com")
not_hot_adam

User(name='Adam', surname='Brzyzek', email='nothotadam@gmail.com')

In [349]:
not_hot_adam.model_dump_json()

'{"name":"Adam","surname":"Brzyzek","email":"nothotadam@gmail.com"}'

In [350]:
not_hot_adam.model_dump()

{'name': 'Adam', 'surname': 'Brzyzek', 'email': 'nothotadam@gmail.com'}

In [351]:
import inspect
from beanie import Document
from pydantic import BaseModel
  

inspect.getmro(Document)

(beanie.odm.documents.Document,
 lazy_model.parser.new.LazyModel,
 pydantic.main.BaseModel,
 beanie.odm.interfaces.setters.SettersInterface,
 beanie.odm.interfaces.inheritance.InheritanceInterface,
 beanie.odm.interfaces.find.FindInterface,
 beanie.odm.interfaces.aggregate.AggregateInterface,
 beanie.odm.interfaces.getters.OtherGettersInterface,
 object)

In [352]:
class User(Document):
    name: str
    surname: str
    email: str

type(User)

pydantic._internal._model_construction.ModelMetaclass

if you run it you will get 'CollectionWasNotInitialized'

first initialize Collection

In [353]:
import os

from beanie import init_beanie
from motor.motor_asyncio import AsyncIOMotorClient


client = AsyncIOMotorClient(os.getenv("MONGODB_URI"))
await init_beanie(
    database=client.workshop,
    document_models=[User],
    multiprocessing_mode=True,
)
client.drop_database(name_or_database=client.workshop)


<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/coolig/.pyenv/versions/3.11.9/lib/python3.11/asyncio/futures.py:387]>

In [354]:
hot_adam = User(name="Adam", surname="Brzyzek", email="hotadam@gmail.com")
hot_adam


User(id=None, revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')

In [355]:
hot_adam.model_dump()

{'id': None,
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com'}

In [356]:
User.insert(hot_adam)

<coroutine object Document.insert at 0x10ecbf340>

coroutine

In [357]:
hot_adam.model_dump()

{'id': None,
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com'}

In [358]:
await User.insert(hot_adam)

User(id=ObjectId('66c7c1d8d4bd4b7536884e09'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')

In [382]:
type(hot_adam.id)
from beanie import PydanticObjectId

Explain revision_id

In [380]:
hot_adam.model_dump()


{'active': True,
 'id': '66c7c1d9d4bd4b7536884e0d',
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com',
 'address': None}

In [361]:
client.drop_database(name_or_database=client.workshop)


<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/coolig/.pyenv/versions/3.11.9/lib/python3.11/asyncio/futures.py:387]>

In [362]:
hot_adam = User(name="Adam", surname="Brzyzek", email="hotadam@gmail.com")
hot_adam
await User.insert_one(hot_adam)


User(id=ObjectId('66c7c1d8d4bd4b7536884e0a'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')

In [363]:
hot_adam.model_dump()

{'id': '66c7c1d8d4bd4b7536884e0a',
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com'}

In [364]:
hot_adam = User(name="Adam", surname="Brzyzek", email="hotadam@gmail.com")
await User.create(hot_adam)
hot_adam.model_dump()

{'id': '66c7c1d9d4bd4b7536884e0b',
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com'}

In [365]:
adam = await User.get(hot_adam.id)
adam, type(adam)

(User(id=ObjectId('66c7c1d9d4bd4b7536884e0b'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 __main__.User)

diffrence between get and find one

In [366]:
await User.find_one(User.id == hot_adam.id)

User(id=ObjectId('66c7c1d9d4bd4b7536884e0b'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')

find more powerfull

In [367]:
result = User.find()
print(result, type(result))

<beanie.odm.queries.find.FindMany object at 0x10ee86310> <class 'beanie.odm.queries.find.FindMany'>


In [368]:
result = User.find({}).to_list()
# print(result)
# print(len(result))


In [369]:
await result

[User(id=ObjectId('66c7c1d8d4bd4b7536884e0a'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 User(id=ObjectId('66c7c1d9d4bd4b7536884e0b'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')]

In [370]:
result = await User.find().first_or_none()
print(result)
print(type(result))

id=ObjectId('66c7c1d8d4bd4b7536884e0a') revision_id=None name='Adam' surname='Brzyzek' email='hotadam@gmail.com'
<class '__main__.User'>


pymongo syntax - ble string XD

In [371]:
await User.find({"_id": hot_adam.id}).to_list()


[User(id=ObjectId('66c7c1d9d4bd4b7536884e0b'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')]

In [372]:
await User.find_many().to_list()


[User(id=ObjectId('66c7c1d8d4bd4b7536884e0a'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 User(id=ObjectId('66c7c1d9d4bd4b7536884e0b'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')]

In [373]:
await User.find_all().to_list()


[User(id=ObjectId('66c7c1d8d4bd4b7536884e0a'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 User(id=ObjectId('66c7c1d9d4bd4b7536884e0b'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')]

In [374]:
class BaseInformation(BaseModel):
  name: str
  surname: str

result = await User.find().project(BaseInformation).first_or_none()
result

BaseInformation(name='Adam', surname='Brzyzek')

Embedeed Documnet

In [377]:
from typing import Optional


class Address(BaseModel):
    country: str
    city: str
    street: str
    building_number: str
    zip_code: str


class User(Document):
  name: str
  surname: str
  email: str
  address: Optional[Address] = None


Use inheritance

In [378]:
class Active(BaseModel):
    active: bool = True

class Address(BaseModel):
    country: str
    city: str
    street: str
    building_number: str
    zip_code: str

class User(Document, Active):
    name: str
    surname: str
    email: str
    address: Optional[Address] = None

client = AsyncIOMotorClient(os.getenv("MONGODB_URI"))

await init_beanie(
    database=client.workshop,
    document_models=[User],
    multiprocessing_mode=True,
)
client.drop_database(name_or_database=client.workshop)

hot_adam = User(name="Adam", surname="Brzyzek", email="hotadam@gmail.com")
await User.insert(hot_adam)

User(active=True, id=ObjectId('66c7c1d9d4bd4b7536884e0d'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com', address=None)

In [379]:
hot_adam.model_dump()


{'active': True,
 'id': '66c7c1d9d4bd4b7536884e0d',
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com',
 'address': None}

### Exercise 1

* create document Task with name, description, priority(low, normal, urgent), Size(S, M, L), Status(Backlog, TODO, InProgress, OnHold, Review, Done)
* add to User Document recently task added by user  
* add one user & task